<a href="https://colab.research.google.com/github/secutron/FixMatch-pytorch/blob/master/FixMatch_pytorch_on_TPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FixMatch with Pytorch on TPUs - experimental

Code: https://github.com/vfdev-5/FixMatch-pytorch/


### Install requirements:

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


try:
  import torch_xla
  import ignite
except ImportError:
  VERSION = "20210304"    
  # VERSION = "nightly"
  # VERSION = "20200607"
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION
  !pip install --upgrade git+https://github.com/pytorch/ignite
  !pip install --upgrade --pre hydra-core
  !pip install wandb
  !pip uninstall -y pillow && CC="cc -mavx2" pip install --no-cache-dir --force-reinstall pillow-simd


### Download dataset:

In [2]:
# Download dataset:
from torchvision.datasets import CIFAR10

CIFAR10("./cifar10", train=True, download=True);

Using downloaded and verified file: ./cifar10/cifar-10-python.tar.gz
Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


### Get the code:

In [3]:
!git clone https://github.com/secutron/FixMatch-pytorch/

fatal: destination path 'FixMatch-pytorch' already exists and is not an empty directory.


#### Optionally, login to `W&B`

To skip logging to `W&B`, please set `online_exp_tracking.wandb=false` 

In [4]:
# !wandb login <token>

### Let's train ResNet18 model in a faster mode

In [5]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python main_fixmatch.py distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true solver.num_epochs=1 \
    ssl.confidence_threshold=0.7 ema_decay=0.9 ssl.cta_update_every=15 solver.optimizer.params.lr=0.1

/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'ssl/cta_pseudo': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'solver/default': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'dataflow/cifar10': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
2021-08-04 07:41:39,440 ignite.distributed.launcher.Parallel INFO: I

### Let's train WRN-28-2 model

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python -u main_fixmatch.py model=WRN-28-2 distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true